Naše úloha bude rozšířením veliceho známého *knapsack problému*, který spočívá v tom, že máme $n$ předmětů, z nichž každý má váhu $w_i$ a cenu $p_i$, a batoh o maximální kapacitě $C$. A našim úkolem je zvolit takové předměty, abychom nepřekročili maximální kapacitu a zároveň abychom maximalizovali honodtu předmětů umístěných v batohu. My tuto úlohu rozšíříme do tzv. *quadratic knapsack problému*, u kterého je rozdíl v tom, že pouze nesčítáme ceny předmětů, ale že různé dvojice předmětů mohou mít nějaké bonusové ohodnocení. 


Tato úloha by šla matematicky popsat následujícím způsobem:
\begin{align}\max \Big\{\sum_{i=1}^n p_ix_i+\sum_{i=1}^n\sum_{j=1,i\neq j}^n P_{ij}x_ix_j: x\in X, x \text{ binární}\Big\},\end{align} kde $X=\Big\{x\in\{0,1\}^n:\sum_{i=1}^n w_ix_i\leq C, x_i\in\{0,1\}$ pro $i\in\widehat{n}\Big\}$, $P_{ij}$ je cena, když jsou v batohu obsaženy předmety $i$ a $j$, a $x_i$ představuje, zdali je $i$-tý předmět obsažen ($x_i=1$) či neobsažen ($x_i=0$) v batohu.

Nyní si představíme heuristiku, kterou bychom rádi použili. Naše heuristika bude využívat tzv. dynamického programování, což je pojem úzce spjatý s Bellmanovou rovnicí a Bellmanovým principem optimality, který nám pomáhá celkový problém řešit pomocí série menších sub-problémů. Našim cílem je tedy maximalizovat hodnotu obsahu batohu za podmínky, že celková hmotnost batohu bude menši nebo rovna maximální kapacitě $C$. Proto si potom pro každé $c$ takové, že $c\geq 0$ a $c\leq C$, definujeme $f(m,c)$, což bude celková hodnota nejvýhodnější kombinace prvních $m$ předmětů s celkovou váhou $c$, tj. \begin{align}
f(m,c)=\max \left\{\sum _{i=1}^{m}p_{i}x_{i}+\sum _{i=1}^{m}\sum _{j=1,i\neq j}^{m}P_{ij}x_{i}x_{j}:\sum _{i=1}^{m}w_{i}=c,1\leq i\leq m\right\}.\end{align} Potom tedy pro $f(\cdot,\cdot)$ mohou nastat pouze dva případy a to \begin{align}{\displaystyle f(m,c)={\begin{cases}\max \big\{f(m-1,c);f(m-1,c-w_{m})+p_{m}+\sum _{i=1}^{m-1}P_{im}x_{i}\big\}&{\text{pokud }}w_{m}\leq c\\f(m-1,c)&{\text{jinak.}}\end{cases}}}\end{align}.  

Tuto heurisitku aplikujeme na tři různé případy, kde první z nich bude nejlehčí a také ručně ověřitelný. U zbylých dvou příkladů by již bylo velice časové náročné hledat jejich řešení standardním způsobem na papíře. 

## 1. příklad

Máme tři předměty s váhami $w_1=w_2=w_3=1$ a maticí cen $P=\begin{pmatrix} 10 & 0 & 0 \\ 0 & 1 & 10 \\ 0 & 10 & 1
\end{pmatrix}$. Zároveň bude maximální kapacita batohu $C=2$.

Díky malému počtu předmětů lze lehce řešit příklad ručně, čimž bychom obdrželi výsledek, že nejlepší je vzít 2. a 3. předmět, jejiž celková hodnota bude 22.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  5 11:41:52 2021

@author: Filip
"""

import numpy as np
import numpy.matlib

# ZADANI: w - vahy, C - kapacita batohu, Q - matice cen

w=np.array([0, 1, 1, 1])
C=2
Q=np.matrix([[10,0,0],[0,1,10],[0,10,1]])

# ALOKACE: f - matice profitu, S - vektor reseni

f=np.full((len(w),C+1),-np.inf)
f[0,0]=0


jednicky=[[-1]]
for j in range(C):
    jednicky1=jednicky.append([-1])
    
S=[[]]*(len(w)+1)
for i in range(0,len(w)+1):
    S[i]=jednicky.copy()
    
# HLAVNI CAST

for k in range(1,len(w)):
    for r in range(0,C+1):
        if f[k-1,r] > f[k,r]:
            f[k,r]=f[k-1,r]
            S[k][r]=S[k-1][r]
        if r+w[k] <= C:
            if S[k-1][r][0]==-1:
                beta=Q[k-1,k-1]
            else:
                SS=np.union1d(S[k-1][r],k)-1
                nuly=np.matlib.zeros((1,len(w)-1))
                nuly[0,SS.astype(int)]=1
                beta=nuly*Q*np.transpose(nuly)
            if beta > f[k,r+w[k]]:
                f[k,r+w[k]]=beta
                if S[k-1][r][0]==-1:
                    S[k][r+w[k]]=[k]
                else:
                    S[k][r+w[k]]=np.union1d(S[k-1][r],(k))
                    
r_star=np.argmax(f)
r_star=r_star%(C+1)
print('Vezmeme predmet(y)', S[len(w)-1][r_star], 'a celkovy profit bude', f[len(w)-1,r_star])

Vezmeme predmet(y) [1 3] a celkovy profit bude 11.0


Ihned můžeme pozorovat, že výsledek naší heuristiky je celkem daleko od optimálního výsledku. Na někoho by její výsledek dokonce mohl působit dojmem, že je tato heuristika "špatná" či příliš nepřesná. Nicméně u tohoto konkrétního příkladu může být její výsledek zavádějicí, protože na první pohled by se mohlo zdát, že její výsledek je polovinou optimálního výsledku, ale pravda je taková, že naše heuristika našla druhý nejlepší výsledek. V takto zadaném příkladu totiž nelze dosáhnout jiného profitu mezi 11 a 22, tudíž druhý nejlepší profit není špatný výsledek. Abychom se utvrdili v kvalitě použité heuristiky, tak se ještě podíváme na dva o něco složitější příklady.

## 2. a 3. příklad

Máme 15 předmětů s vektorem vah $w=(13,14,14,15,15,9,26,24,13,11,9,12,25,12,26)$ a maticí cen $P=\begin{pmatrix}
    7 &   12 &    7 &    6 &   13 &    8 &   11 &    7 &   15 &   23 &   14 &   15 &   17 &    9 &   15\\
   12 &    6 &   15 &   13 &   10 &   15 &    9 &   10 &    8 &   17 &   11 &   13 &   12 &   16 &   15\\
    7 &   15 &   13 &   11 &   16 &    6 &    8 &   14 &   13 &    4 &   14 &    8 &   15 &    9 &   16\\
    6 &   13 &   11 &   16 &   10 &   13 &   14 &   14 &   17 &   15 &   14 &    6 &   24 &   13 &    4\\
   13 &   10 &   16 &   10 &    5 &    9 &    7 &   25 &   12 &    6 &    6 &   16 &   10 &   15 &   14\\
    8 &   15 &    6 &   13 &    9 &   10 &    2 &   13 &   12 &   16 &    9 &   11 &   23 &   10 &   21\\
   11 &    9 &    8 &   14 &    7 &    2 &    9 &    8 &   18 &    4 &   13 &   14 &   14 &   17 &   15\\
    7 &   10 &   14 &   14 &   25 &   13 &    8 &   23 &    9 &   16 &   12 &    3 &   14 &   14 &   27\\
   15 &    8 &   13 &   17 &   12 &   12 &   18 &    9 &   18 &   15 &   16 &   13 &   14 &    7 &   17\\
   23 &   17 &    4 &   15 &    6 &   16 &    4 &   16 &   15 &   12 &   28 &    5 &   19 &    6 &   18\\
   14 &   11 &   14 &   14 &    6 &    9 &   13 &   12 &   16 &   28 &    9 &   13 &    4 &   13 &   16\\
   15 &   13 &    8 &    6 &   16 &   11 &   14 &    3 &   13 &    5 &   13 &   22 &   11 &   19 &   13\\
   17 &   12 &   15 &   24 &   10 &   23 &   14 &   14 &   14 &   19 &    4 &   11 &   17 &   15 &   12\\
    9 &   16 &    9 &   13 &   15 &   10 &   17 &   14 &    7 &    6 &   13 &   19 &   15 &   32 &   16\\
   15 &   15 &   16 &    4 &   14 &   21 &   15 &   27 &   17 &   18 &   16 &   13 &   12 &   16 &    8
\end{pmatrix}.$ 

Zároveň budeme předpokládat maximální kapacity batohu $C_1=22$ a $C_2=50$.

Bohužel si již v těchto příkladech nemůžeme ručně ověřit správná řešení, nicméně řešení by měla být předměty 10 a 11 s celkovou hodnotou 77 a předměty 1, 9, 10 a 11 s celkovou hodnotou 268.

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  5 13:12:42 2021

@author: Filip
"""

import numpy as np
import numpy.matlib

# ZADANI: w - vahy, C - kapacita batohu, Q - matice cen

w=np.array([0, 13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
C=22
Q=np.array([
7	,12	,7	,6	,13	,8	,11	,7	,15	,23	,14	,15	,17	,9	,15,
12	,6	,15	,13	,10	,15	,9	,10	,8	,17	,11	,13	,12	,16	,15,
7	,15	,13	,11	,16	,6	,8	,14	,13	,4	,14	,8	,15	,9	,16,
6	,13	,11	,16	,10	,13	,14	,14	,17	,15	,14	,6	,24	,13	, 4,
13	,10	,16	,10	,5	,9	,7	,25	,12	,6	,6	,16	,10	,15	,14,
8	,15	,6	,13	,9	,10	,2	,13	,12	,16	,9	,11	,23	,10	,21,
11	,9	,8	,14	,7	,2	,9	,8	,18	,4	,13	,14	,14	,17	,15,
7	,10	,14	,14	,25	,13	,8	,23	,9	,16	,12	,3	,14	,14	,27,
15	,8	,13	,17	,12	,12	,18	,9	,18	,15	,16	,13	,14	,7	,17,
23	,17	,4	,15	,6	,16	,4	,16	,15	,12	,28	,5	,19	,6	,18,
14	,11	,14	,14	,6	,9	,13	,12	,16	,28	,9	,13	,4	,13	,16,
15	,13	,8	,6	,16	,11	,14	,3	,13	,5	,13	,22	,11	,19	,13,
17	,12	,15	,24	,10	,23	,14	,14	,14	,19	,4	,11	,17	,15	,12,
9	,16	,9	,13	,15	,10	,17	,14	,7	,6	,13	,19	,15	,32	,16,
15	,15	,16	,4	,14	,21	,15	,27	,17	,18	,16	,13	,12	,16	, 8
    ])

Q=Q.reshape(15,15)

f=np.full((len(w),C+1),-np.inf)
f[0,0]=0

jednicky=[[-1]]
for j in range(C):
    jednicky1=jednicky.append([-1])

S=[[]]*len(w)
for i in range(0,len(w)):
    S[i]=jednicky.copy()

for k in range(1,len(w)):
    for r in range(0,C+1):
        if f[k-1,r] > f[k,r]:
            f[k,r]=f[k-1,r]
            S[k][r]=S[k-1][r]
        if r+w[k] <= C:
            if S[k-1][r][0]==-1:
                beta=Q[k-1,k-1]
            else:
                SS=np.union1d(S[k-1][r],k)-1
                nuly=np.matlib.zeros((1,len(w)-1))
                nuly[0,SS.astype(int)]=1
                beta=nuly*Q*np.transpose(nuly)
            if beta > f[k,r+w[k]]:
                f[k,r+w[k]]=beta
                if S[k-1][r][0]==-1:
                    S[k][r+w[k]]=[k]
                else:
                    S[k][r+w[k]]=np.union1d(S[k-1][r],(k))
                    
r_star=np.argmax(f)
r_star=r_star%(C+1)
print('Vezmeme predmety', S[len(w)-1][r_star], 'a celkovy profit bude', f[len(w)-1,r_star])

Vezmeme predmety [10 11] a celkovy profit bude 77.0


In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  5 13:12:42 2021

@author: Filip
"""

import numpy as np
import numpy.matlib
#import array_to_latex as a2l

# ZADANI: w - vahy, C - kapacita batohu, Q - matice cen

w=np.array([0, 13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
C=50
Q=np.array([
7	,12	,7	,6	,13	,8	,11	,7	,15	,23	,14	,15	,17	,9	,15,
12	,6	,15	,13	,10	,15	,9	,10	,8	,17	,11	,13	,12	,16	,15,
7	,15	,13	,11	,16	,6	,8	,14	,13	,4	,14	,8	,15	,9	,16,
6	,13	,11	,16	,10	,13	,14	,14	,17	,15	,14	,6	,24	,13	, 4,
13	,10	,16	,10	,5	,9	,7	,25	,12	,6	,6	,16	,10	,15	,14,
8	,15	,6	,13	,9	,10	,2	,13	,12	,16	,9	,11	,23	,10	,21,
11	,9	,8	,14	,7	,2	,9	,8	,18	,4	,13	,14	,14	,17	,15,
7	,10	,14	,14	,25	,13	,8	,23	,9	,16	,12	,3	,14	,14	,27,
15	,8	,13	,17	,12	,12	,18	,9	,18	,15	,16	,13	,14	,7	,17,
23	,17	,4	,15	,6	,16	,4	,16	,15	,12	,28	,5	,19	,6	,18,
14	,11	,14	,14	,6	,9	,13	,12	,16	,28	,9	,13	,4	,13	,16,
15	,13	,8	,6	,16	,11	,14	,3	,13	,5	,13	,22	,11	,19	,13,
17	,12	,15	,24	,10	,23	,14	,14	,14	,19	,4	,11	,17	,15	,12,
9	,16	,9	,13	,15	,10	,17	,14	,7	,6	,13	,19	,15	,32	,16,
15	,15	,16	,4	,14	,21	,15	,27	,17	,18	,16	,13	,12	,16	, 8
    ])

Q=Q.reshape(15,15)

# ALOKACE: f - matice profitu, S - vektor reseni

f=np.full((len(w),C+1),-np.inf)
f[0,0]=0

jednicky=[[-1]]
for j in range(C):
    jednicky1=jednicky.append([-1])
    
S=[[]]*len(w+1)
for i in range(0,len(w)):
    S[i]=jednicky.copy()
    
# HLAVNI CAST

for k in range(1,len(w)):
    for r in range(0,C+1):
        if f[k-1,r] > f[k,r]:
            f[k,r]=f[k-1,r]
            S[k][r]=S[k-1][r]
        if r+w[k] <= C:
            if S[k-1][r][0]==-1:
                beta=Q[k-1,k-1]
            else:
                SS=np.union1d(S[k-1][r],k)-1
                nuly=np.matlib.zeros((1,len(w)-1))
                nuly[0,SS.astype(int)]=1
                beta=nuly*Q*np.transpose(nuly)
            if beta > f[k,r+w[k]]:
                f[k,r+w[k]]=beta
                if S[k-1][r][0]==-1:
                    S[k][r+w[k]]=[k]
                else:
                    S[k][r+w[k]]=np.union1d(S[k-1][r],(k))
                    
r_star=np.argmax(f)
r_star=r_star%(C+1)
print('Vezmeme predmety', S[len(w)-1][r_star], 'a celkovy profit bude', f[len(w)-1,r_star])

Vezmeme predmety [ 4  9 10 11] a celkovy profit bude 265.0


Jak můžeme pozorovat, tak u kapacity $C_1=22$ jsme dostali optimální výsledek a u kapacity $C_2=50$ jsme dostali druhý nejlepší výsledek. Pozorujeme tedy, že naše heuristika nám dává opravdu dobré výsledky.   

Tuto naši heuristiku ještě porovnáme s jednou ze základních heuristik a to s Random shooting heurisitkou, která bude lehce upravena pro náš příklad. Tato heurisitka bude jednoduše spočívat v tom, že budeme do batohu dávat náhodně různé předměty při dodržení maximální kapacity.

In [18]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 10 18:25:37 2021

@author: Filip
"""

import numpy as np
import numpy.matlib
import random

w=np.array([0, 1, 1, 1])
# pridelime kazdemu predmetu cislo, aby nedoslo k zamene predmetu se stejnou vahou
numbers=np.arange(0,len(w),1)
w=np.c_[w, numbers].T 
C=2

Q=np.matrix([[10,0,0],[0,1,10],[0,10,1]])

evals=200

#results=np.empty(shape=(2,evals)).tolist()

best_profit=0
best_S=np.empty( shape=(1, 0) )


for j in range(0,evals):

    S=np.empty( shape=(1, 0) )
    new_w=np.array([[0,1],[0,1]])
    
    
    while C >= np.amin(new_w[0,1:]):
    
        new_w=np.array([[0],[0]])
        # Vyfiltrujeme veci, ktere se do batohu vejdou
        for k in range(1,len(w[0])):
            if w[0,k] <= C:
                new_w=np.hstack((new_w, np.atleast_2d(w[:,k]).T))
                
        
        # Nahodne vyberem index z new_w
        if len(new_w[0])-1 == 0:
            break
        ind=random.randint(1,len(new_w[0])-1)
        
        C=C-new_w[0,ind]
        S=np.append(S, new_w[1,ind])
        w=np.delete(w,np.where(w[1,:] == new_w[1,ind]),1)
    
    w=np.array([0, 1, 1, 1])
    nuly=np.matlib.zeros((1,len(w)-1))
    nuly[0,S.astype(int)-1]=1
    beta=int(nuly*Q*np.transpose(nuly))
    #results[0][j]=S
    #results[1][j]=beta
    if beta > best_profit:
        best_profit=beta
        best_S=S
    w=np.c_[w, numbers].T 
    C=2


print('Vezmeme predmety', best_S, 'a celkovy profit bude', best_profit)

Vezmeme predmety [3. 2.] a celkovy profit bude 22


In [19]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 11 20:58:38 2021

@author: Filip
"""


import numpy as np
import numpy.matlib
import random

w=np.array([0, 13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
# pridelime kazdemu predmetu cislo, aby nedoslo k zamene predmetu se stejnou vahou
numbers=np.arange(0,len(w),1)
w=np.c_[w, numbers].T 
C=22
Q=np.array([
7	,12	,7	,6	,13	,8	,11	,7	,15	,23	,14	,15	,17	,9	,15,
12	,6	,15	,13	,10	,15	,9	,10	,8	,17	,11	,13	,12	,16	,15,
7	,15	,13	,11	,16	,6	,8	,14	,13	,4	,14	,8	,15	,9	,16,
6	,13	,11	,16	,10	,13	,14	,14	,17	,15	,14	,6	,24	,13	, 4,
13	,10	,16	,10	,5	,9	,7	,25	,12	,6	,6	,16	,10	,15	,14,
8	,15	,6	,13	,9	,10	,2	,13	,12	,16	,9	,11	,23	,10	,21,
11	,9	,8	,14	,7	,2	,9	,8	,18	,4	,13	,14	,14	,17	,15,
7	,10	,14	,14	,25	,13	,8	,23	,9	,16	,12	,3	,14	,14	,27,
15	,8	,13	,17	,12	,12	,18	,9	,18	,15	,16	,13	,14	,7	,17,
23	,17	,4	,15	,6	,16	,4	,16	,15	,12	,28	,5	,19	,6	,18,
14	,11	,14	,14	,6	,9	,13	,12	,16	,28	,9	,13	,4	,13	,16,
15	,13	,8	,6	,16	,11	,14	,3	,13	,5	,13	,22	,11	,19	,13,
17	,12	,15	,24	,10	,23	,14	,14	,14	,19	,4	,11	,17	,15	,12,
9	,16	,9	,13	,15	,10	,17	,14	,7	,6	,13	,19	,15	,32	,16,
15	,15	,16	,4	,14	,21	,15	,27	,17	,18	,16	,13	,12	,16	, 8
    ])

Q=Q.reshape(15,15)
evals=200

#results=np.empty(shape=(2,evals)).tolist()

best_profit=0
best_S=np.empty( shape=(1, 0) )


for j in range(0,evals):

    S=np.empty( shape=(1, 0) )
    new_w=np.array([[0,1],[0,1]])
    
    
    while C >= np.amin(new_w[0,1:]):
    
        new_w=np.array([[0],[0]])
        # Vyfiltrujeme veci, ktere se do batohu vejdou
        for k in range(1,len(w[0])):
            if w[0,k] <= C:
                new_w=np.hstack((new_w, np.atleast_2d(w[:,k]).T))
                
        
        # Nahodne vyberem index z new_w
        if len(new_w[0])-1 == 0:
            break
        ind=random.randint(1,len(new_w[0])-1)
        
        C=C-new_w[0,ind]
        S=np.append(S, new_w[1,ind])
        w=np.delete(w,np.where(w[1,:] == new_w[1,ind]),1)
    
    w=np.array([0, 13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
    nuly=np.matlib.zeros((1,len(w)-1))
    nuly[0,S.astype(int)-1]=1
    beta=int(nuly*Q*np.transpose(nuly))
    #results[0][j]=S
    #results[1][j]=beta
    if beta > best_profit:
        best_profit=beta
        best_S=S
    w=np.c_[w, numbers].T 
    C=22

print('Vezmeme predmety', best_S, 'a celkovy profit bude', best_profit)

Vezmeme predmety [10. 11.] a celkovy profit bude 77


In [39]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 10 17:32:40 2021

@author: Filip
"""

import numpy as np
import numpy.matlib
import random

w=np.array([0, 13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
# pridelime kazdemu predmetu cislo, aby nedoslo k zamene predmetu se stejnou vahou
numbers=np.arange(0,len(w),1)
w=np.c_[w, numbers].T 
C=50
Q=np.array([
7	,12	,7	,6	,13	,8	,11	,7	,15	,23	,14	,15	,17	,9	,15,
12	,6	,15	,13	,10	,15	,9	,10	,8	,17	,11	,13	,12	,16	,15,
7	,15	,13	,11	,16	,6	,8	,14	,13	,4	,14	,8	,15	,9	,16,
6	,13	,11	,16	,10	,13	,14	,14	,17	,15	,14	,6	,24	,13	, 4,
13	,10	,16	,10	,5	,9	,7	,25	,12	,6	,6	,16	,10	,15	,14,
8	,15	,6	,13	,9	,10	,2	,13	,12	,16	,9	,11	,23	,10	,21,
11	,9	,8	,14	,7	,2	,9	,8	,18	,4	,13	,14	,14	,17	,15,
7	,10	,14	,14	,25	,13	,8	,23	,9	,16	,12	,3	,14	,14	,27,
15	,8	,13	,17	,12	,12	,18	,9	,18	,15	,16	,13	,14	,7	,17,
23	,17	,4	,15	,6	,16	,4	,16	,15	,12	,28	,5	,19	,6	,18,
14	,11	,14	,14	,6	,9	,13	,12	,16	,28	,9	,13	,4	,13	,16,
15	,13	,8	,6	,16	,11	,14	,3	,13	,5	,13	,22	,11	,19	,13,
17	,12	,15	,24	,10	,23	,14	,14	,14	,19	,4	,11	,17	,15	,12,
9	,16	,9	,13	,15	,10	,17	,14	,7	,6	,13	,19	,15	,32	,16,
15	,15	,16	,4	,14	,21	,15	,27	,17	,18	,16	,13	,12	,16	, 8
    ])

Q=Q.reshape(15,15)
evals=1000

#results=np.empty(shape=(2,evals)).tolist()

best_profit=0
best_S=np.empty( shape=(1, 0) )


for j in range(0,evals):

    S=np.empty( shape=(1, 0) )
    new_w=np.array([[0,1],[0,1]])
    
    
    while C >= np.amin(new_w[0,1:]):
    
        new_w=np.array([[0],[0]])
        # Vyfiltrujeme veci, ktere se do batohu vejdou
        for k in range(1,len(w[0])):
            if w[0,k] <= C:
                new_w=np.hstack((new_w, np.atleast_2d(w[:,k]).T))
                
        
        # Nahodne vyberem index z new_w
        if len(new_w[0])-1 == 0:
            break
        ind=random.randint(1,len(new_w[0])-1)
        
        C=C-new_w[0,ind]
        S=np.append(S, new_w[1,ind])
        w=np.delete(w,np.where(w[1,:] == new_w[1,ind]),1)
    
    w=np.array([0, 13, 14, 14, 15, 15, 9, 26, 24, 13, 11, 9, 12, 25, 12, 26])
    nuly=np.matlib.zeros((1,len(w)-1))
    nuly[0,S.astype(int)-1]=1
    beta=int(nuly*Q*np.transpose(nuly))
    #results[0][j]=S
    #results[1][j]=beta
    if beta > best_profit:
        best_profit=beta
        best_S=S
    w=np.c_[w, numbers].T 
    C=50

print('Vezmeme predmety', best_S, 'a celkovy profit bude', best_profit)

Vezmeme predmety [10. 11.  9.  1.] a celkovy profit bude 268


Ačkoli se jedná o vcelku jednoduchou heuristiku, u které by se dalo říct, že používá na řešení hrubou sílu, tak její výsledky předčily naši původní heuristiku, protože dosáhala ve všech třech příkladech optimálního řešení. Nicméně to není důvod pro to, abychom naši původní heuristiku zavrhli, protože její síla by se pravděpodobně ukázala při řádově vyšších množstvích předmetů. 

Na závěr bychom uvedli, že obě námi testované heuristiky si dobře vedly s naší úlohou. Pro užití bychom doporučili používat při nízkých počtech předmětů Random shooting heuristiku, ale při vyšších počtech již naši hlavní heuristiku.